In [179]:
import numpy as np
from typing import List
import random
from collections import defaultdict, Counter
import re

### Q1: Chance that it rains more than n (e.g. 100) days in Vancouver

#### 1.1 Code implementation

In [163]:
# Using Monte-Carlo simulation
def prob_rain_more_than_n(p: Sequence[float], n: int) -> float:
    total_days = len(p)
    trials = 10000
    
    # Run multiple simulations
    simulations = np.random.rand(trials, total_days) < p  # Generate matrix with random weather of [size trials X total_days]
    rainy_days_per_trial = np.sum(simulations, axis=1) # Count rainy days per trial
    
    # Count how many trials had more than n rainy days
    successful_trials = np.sum(rainy_days_per_trial > n)
    
    # Return the fraction of successful trials as the probability
    return successful_trials / trials

#### 1.2 Test cases

In [168]:
# Function to generate a list of probabilities between 0.1 and 0.5
def generate_probabilities(min_value: float, max_value: float, days: int) -> List[float]:
    return [random.uniform(min_value, max_value) for _ in range(days)]

# Test case 1: Probability of rain is 0.3 on each day
p_1 = [0.3] * 365
n_1 = 100
result_1 = prob_rain_more_than_n(p_1, n_1)
print(f"Test Case 1 - Probability of more than {n_1} rainy days: {result_1:.6f}")

# Test case 2: Probability of rain varies between 0.1 and 0.9
p_2 = [0.1, 0.9] * (365 // 2) + [0.5]  # alternating 0.1 and 0.9, with last day 0.5
n_2 = 180
result_2 = prob_rain_more_than_n(p_2, n_2)
print(f"Test Case 2 - Probability of more than {n_2} rainy days: {result_2:.6f}")

# Test case 3: Probability of rain varies between 0.1 and 0.5
p_3 = generate_probabilities(0.1, 0.5, 365)  # 365 probabilities between 0.1 and 0.5
n_3 = 100
result_3 = prob_rain_more_than_n(p_3, n_3)
print(f"Test Case 3 - Probability of more than {n_3} rainy days: {result_3:.6f}")



Test Case 1 - Probability of more than 100 rainy days: 0.842900
Test Case 2 - Probability of more than 180 rainy days: 0.636600
Test Case 3 - Probability of more than 100 rainy days: 0.885400


### Q2: All combinations of the words that can produce a word sequence from the given sequence of phonemes

#### 2.1 Code Implementation

In [107]:
# Sample pronunciation dictionary
pronunciation_dict = {
    "ABACUS": ["AE", "B", "AH", "K", "AH", "S"],
    "BOOK": ["B", "UH", "K"],
    "THEIR": ["DH", "EH", "R"],
    "THERE": ["DH", "EH", "R"],
    "TOMATO_1": ["T", "AH", "M", "AA", "T", "OW"],
    "TOMATO_2": ["T", "AH", "M", "EY", "T", "OW"]
}

In [111]:
# Preprocess the dictionary to create a phoneme-to-words map
def preprocess_pronunciation_dict(pronunciation_dict):
    phoneme_to_words = defaultdict(list)
    for word, phonemes in pronunciation_dict.items():
        phoneme_tuple = tuple(phonemes)
        phoneme_to_words[phoneme_tuple].append(word)
    return phoneme_to_words

# Function to find all combinations of words with the given phoneme sequence
def find_word_combos_with_pronunciation(phonemes: Sequence[str]) -> List[Sequence[str]]:
    phoneme_to_words = preprocess_pronunciation_dict(pronunciation_dict)
    
    # Recursive function to find combinations
    def find_combinations(start):
        if start == len(phonemes):
            return [[]]  # If we reach the end, return an empty list as the base case
        results = []
        for length in range(1, len(phonemes) - start + 1):
            sub_phonemes = tuple(phonemes[start:start + length])
            if sub_phonemes in phoneme_to_words:
                for word in phoneme_to_words[sub_phonemes]:
                    for rest in find_combinations(start + length):
                        results.append([word] + rest)
        return results
    
    return find_combinations(0)

#### 2.2 Test cases

In [116]:
# Example usage: exact match
phonemes_1 = ["DH", "EH", "R", "DH", "EH", "R"]
combinations_1 = find_word_combos_with_pronunciation(phonemes_1)
for combo in combinations_1:
    print(combo)

['THEIR', 'THEIR']
['THEIR', 'THERE']
['THERE', 'THEIR']
['THERE', 'THERE']


In [114]:
# Example usage: no match
phonemes_2 = ["X", "Y", "Z"]
combinations_2 = find_word_combos_with_pronunciation(phonemes_2)

if combinations_2:
    for combo in combinations_2:
        print(combo)
else:
    print("No words match the phoneme given !")

No words match the phoneme given !


In [128]:
# Example usage: exact match
phonemes_3 = ["T", "AH", "M", "AA", "T", "OW", "T", "AH", "M", "EY", "T", "OW"]
combinations_3 = find_word_combos_with_pronunciation(phonemes_3)

if combinations_3:
    for combo in combinations_3:
        print(combo)
else:
    print("No words match the phoneme given !")

['TOMATO_1', 'TOMATO_2']


### Q4: CTC implementation according to the paper titled "CTCModel: a Keras Model for Connectionist Temporal Classification"

#### 4.1 Code implementation

In [159]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K

In [160]:
# Helper function for CTC loss
def ctc_loss_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Helper function for CTC decoding
def ctc_decode_lambda_func(args, greedy=True, beam_width=100, top_paths=1):
    y_pred, input_length = args
    decoded, _ = K.ctc_decode(y_pred, input_length, greedy=greedy, beam_width=beam_width, top_paths=top_paths)
    return [tf.sparse.to_dense(seq) for seq in decoded]

# Helper function for LER and SER calculation
def ctc_metrics_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    
    # Decode predictions with CTC decoding
    decoded, _ = K.ctc_decode(y_pred, input_length, greedy=True)
    
    # Convert labels and predictions to sparse tensors for edit distance calculation
    label_sparse = tf.cast(tf.keras.backend.ctc_label_dense_to_sparse(labels, label_length), dtype=tf.int64)
    pred_sparse = tf.cast(decoded[0], dtype=tf.int64)
    
    # Calculate edit distance for LER and SER
    edit_distances = tf.edit_distance(pred_sparse, label_sparse, normalize=True)
    ler = tf.reduce_mean(edit_distances)  # Label Error Rate (LER)
    ser = tf.reduce_mean(tf.cast(edit_distances > 0, tf.float32))  # Sequence Error Rate (SER)
    
    # Return LER and SER as a list of tensors
    return [ler, ser]

In [161]:
class CTCModel:
    def __init__(self, input_shape, num_classes):
        self.model_train = None
        self.model_predict = None
        self.model_eval = None
        
        self.num_classes = num_classes
        self.input_shape = input_shape
        self.build_model()

    def get_model_train(self):
        # Model used for training
        return self.model_train

    def get_model_predict(self):
        # Model used for testing
        return self.model_predict

    def get_model_eval(self):
        # Model used for evaluating
        return self.model_eval

    def build_model(self):
        # Input layer for the images (or sequences)
        self.inputs = layers.Input(shape=self.input_shape, name='input')

        # Add layers: for instance, BiLSTM for recurrent neural networks
        x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(self.inputs)
        x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)

        # Output softmax layer (num_classes + 1 for the blank token)
        self.outputs = layers.Dense(self.num_classes + 1, activation='softmax', name='output')(x)

        # Inputs for CTC loss: true labels, input lengths, and label lengths
        labels = layers.Input(shape=(None,), dtype='int32', name='labels')
        input_length = layers.Input(shape=(1,), dtype='int32', name='input_length')
        label_length = layers.Input(shape=(1,), dtype='int32', name='label_length')

        # Compute the CTC loss using a Lambda layer
        ctc_loss = layers.Lambda(ctc_loss_lambda_func, output_shape=(1,), name='ctc_loss')(
            [self.outputs, labels, input_length, label_length])

        # Training model: takes inputs, labels, input lengths, and label lengths
        self.model_train = Model(inputs=[self.inputs, labels, input_length, label_length], outputs=ctc_loss)

        # Prediction model: used to predict sequences
        self.model_predict = Model(inputs=self.inputs, outputs=self.outputs)

        # Evaluation model: calculates LER and SER
        ler_ser = layers.Lambda(ctc_metrics_lambda_func, name="ctc_metrics")(
            [self.outputs, labels, input_length, label_length])
        self.model_evaluate = Model(inputs=[self.inputs, labels, input_length, label_length], outputs=ler_ser)

    def compile(self, optimizer):
        self.model_train.compile(optimizer=optimizer, loss={'ctc_loss': lambda y_true, y_pred: y_pred})

    def train(self, x, y, input_lengths, label_lengths, batch_size=32, epochs=10):
        self.model_train.fit([x, y, input_lengths, label_lengths], y=None, batch_size=batch_size, epochs=epochs)

    def predict(self, x, input_lengths, greedy=True, beam_width=100, top_paths=1):
        # Get raw predictions from the model
        y_pred = self.model_predict.predict(x)
        
        # Decode predictions with CTC decoding
        decoded_sequences = ctc_decode_lambda_func([y_pred, input_lengths], greedy=greedy, beam_width=beam_width, top_paths=top_paths)
        
        return decoded_sequences

    def evaluate(self, x, y, input_lengths, label_lengths):
        # Evaluate the model on some dataset, calculating LER and SER
        ler, ser = self.model_evaluate.predict([x, y, input_lengths, label_lengths])
        print(f"Label Error Rate (LER): {ler}, Sequence Error Rate (SER): {ser}")
        return ler, ser


## Additional question
### Q3: N most common words in Shakespeare dataset
#### I implemented this in Python since I'm more comfortable with Python as compared to C. 

In [184]:
# Shakespeare text file
file_path = "data/shakespeare.txt"

# Initialize a Counter for word frequencies
word_counts = Counter()

with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        # Convert line to lowercase
        line = line.lower()
        
        # Remove special characters and split into words
        words = re.findall(r'\b\w+\b', line)
        
        # Update word counts
        word_counts.update(words)


In [185]:
# Example usage: 10 most common words
n = 10

most_common_words = word_counts.most_common(n)

print("Most frequent words:")
for word, freq in most_common_words:
    print(f"{word}: {freq}")

Most frequent words:
the: 6287
and: 5690
i: 5111
to: 4934
of: 3760
you: 3211
my: 3120
a: 3018
that: 2664
in: 2403
